In [1]:
%%capture
%pip install llama-index llama-index-embeddings-openai qdrant-client llama-index-vector-stores-qdrant llama-index llama-index-llms-openai llama-index-vector-stores-faiss faiss-cpu llama-index-llms-anthropic tavily-python

In [2]:
import os
import nest_asyncio
from getpass import getpass
from dotenv import load_dotenv

load_dotenv(r"C:\Users\anteb\PycharmProjects\JupyterProject\.env")
nest_asyncio.apply()

CO_API_KEY = os.environ.get('CO_API_KEY') or getpass("Enter CO_API_KEY: ")
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY') or getpass("Enter OPENAI_API_KEY: ")
QDRANT_URL = os.environ.get('QDRANT_URL') or getpass("Enter QDRANT_URL: ")
QDRANT_API_KEY = os.environ.get('QDRANT_API_KEY') or getpass("Enter QDRANT_API_KEY: ")
TAVILY_API_KEY = os.environ.get('TAVILY_API_KEY') or getpass("Enter TAVILY_API_KEY: ")

In [30]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

llm = OpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0.5, max_tokens=512)


In [4]:
from tavily import AsyncTavilyClient

# note the type annotations for the incoming query and the return string
async def search_web(query: str) -> str:
    """Useful for using the web to answer questions."""
    client = AsyncTavilyClient(api_key=TAVILY_API_KEY)
    return str(await client.search(query))

In [26]:
from llama_index.core.agent.workflow import AgentWorkflow

workflow = AgentWorkflow.from_tools_or_functions(
    [search_web],
    system_prompt="You are a helpful assistant that answers questions. If you don't know the answer, you can search the web for information.",
)

In [27]:
response = await workflow.run(user_msg="What is the weather in Smolensk Russia?")
print(str(response))

The current weather in Smolensk, Russia, is as follows:

- **Temperature**: 7.7°C (45.9°F)
- **Condition**: Sunny
- **Wind**: 4.3 mph (6.8 kph) from the northwest
- **Humidity**: 70%
- **Visibility**: 10 km
- **Pressure**: 1017 mb

For more details, you can check the weather on [WeatherAPI](https://www.weatherapi.com/) or [AccuWeather](https://www.accuweather.com/en/ru/smolensk/295475/weather-forecast/295475).


In [8]:
from llama_index.core.workflow import Context

# configure a context to work with our workflow
ctx = Context(workflow)

response = await workflow.run(
    user_msg="My name is Daniil, nice to meet you!", ctx=ctx # give the configured context to the workflow
)
print(str(response))

Nice to meet you, Daniil! How can I assist you today?


In [9]:
from llama_index.core.workflow import JsonPickleSerializer, JsonSerializer

# convert our Context to a dictionary object
ctx_dict = ctx.to_dict(serializer=JsonSerializer())

# create a new Context from the dictionary
restored_ctx = Context.from_dict(
    workflow, ctx_dict, serializer=JsonSerializer()
)

In [10]:
response = await workflow.run(
    user_msg="Do you still remember my name?", ctx=restored_ctx
)
print(str(response))

Yes, I remember your name, Daniil! How can I help you today?


In [ ]:
from llama_index.core.agent.workflow import (
    AgentInput,
    AgentOutput,
    ToolCall,
    ToolCallResult,
    AgentStream,
)

handler = workflow.run(user_msg="What is the weather in Smolenks? Call me by name when you anser", ctx=restored_ctx)


async for event in handler.stream_events():
    if isinstance(event, AgentInput):
       print("Agent input: ", event.input)  # the current input messages
       print("Agent name:", event.current_agent_name)  # the current agent name
    elif isinstance(event, AgentOutput):
       print("Agent output: ", event.response)  # the current full response
       print("Tool calls made: ", event.tool_calls)  # the selected tool calls, if any
       print("Raw LLM response: ", event.raw)  # the raw llm api response
    elif isinstance(event, ToolCallResult):
       print("Tool called: ", event.tool_name)  # the tool name
       print("Arguments to the tool: ", event.tool_kwargs)  # the tool kwargs
       print("Tool output: ", event.tool_output)  # the tool output

## Tools and State

Tools can also be defined that have access to the workflow context. This means you can set and retrieve variables from the context and use them in the tool or between tools.

`AgentWorkflow` uses a context variable called `state` that gets passed to every agent. You can rely on information in `state` being available without explicitly having to pass it in.

**Note:** To access the `Context`, the Context parameter should be the first parameter of the tool.

In [31]:
from llama_index.core.workflow import Context

async def set_name(ctx: Context, name: str) -> str:
    state = await ctx.get("state")
    state["name"] = name
    await ctx.set("state", state)
    return f"Name set to {name}"


workflow = AgentWorkflow.from_tools_or_functions(
    [set_name],
    llm=llm,
    system_prompt="You are a helpful assistant that can set a name.",
    initial_state={"name": "unset"},
)

ctx = Context(workflow)

response = await workflow.run(user_msg="My name is Daniil", ctx=ctx)
print(str(response))

state = await ctx.get("state")
print("Name as stored in state: ",state["name"])

Your name has been set to Daniil.
Name as stored in state:  Daniil


## Human in the Loop

Tools can also be defined that involve a human in the loop. This is useful for tasks that require human input, such as confirming a tool call or providing feedback.

As we'll see in the next section, the way `Workflow`s work under the hood of `AgentWorkflow` is by running `step`s which both emit and receive events. Here's a diagram of the steps (in blue) that makes up an AgentWorkflow and the events (in green) that pass data between them. You'll recognize these events, they're the same ones we were handling in the output stream earlier.

 To get a human in the loop, we'll get our tool to emit an event that isn't received by any other step in the workflow. We'll then tell our tool to wait until it receives a specific "reply" event.

This is a very common pattern for human in the loop, so we have built-in `InputRequiredEvent` and `HumanResponseEvent` events to use for this purpose. If you want to capture different forms of human input, you can subclass these events to match your own preferences.

In [32]:
from llama_index.core.workflow import (
    Context,
    InputRequiredEvent,
    HumanResponseEvent,
)

# a tool that performs a dangerous task
async def dangerous_task(ctx: Context) -> str:
    """A dangerous task that requires human confirmation."""

    # emit an event to the external stream to be captured
    ctx.write_event_to_stream(
        InputRequiredEvent(
            prefix="Are you sure you want to proceed? ",
            user_name="Daniil",
        )
    )

    # wait until we see a HumanResponseEvent
    response = await ctx.wait_for_event(
        HumanResponseEvent, requirements={"user_name": "Daniil"}
    )

    # act on the input from the event
    if response.response.strip().lower() == "yes":
        return "Dangerous task completed successfully."
    else:
        return "Dangerous task aborted."


workflow = AgentWorkflow.from_tools_or_functions(
    [dangerous_task],
    llm=llm,
    system_prompt="You are a helpful assistant that can perform dangerous tasks.",
)